# LAB 1 MOS: Modelado Matematico y Fundamentos en Pyomo

- Samuel Augusto Hernandez 202213772
- Tomas Alberto Rodriguez 2022128686

## Problema 1:

### Parte A:
#### Indentificacion del problema:
El scrum master busca ´maximizar´ la cantidad de tareas segun la prioridad que un grupo de 4 desarrolladores pueden realizar por sprint sin sobrepasar un total de 52 puntos y teniendo en cuenta que cada tarea tiene puntos determinados distintos.
#### Suposiciones:
- La suma de los puntos de historia de las tareas a hacer supera siempre los 52 puntos, es decir, no se podran escoger todas las tareas.
- Los desarrolladores trabajarn siempre en condiciones normales, es decir, sin fatiga u otros factores humanos.
- Ninguna tarea supera los 52 puntos por si sola.
- No ocurrira que todas las tareas tenga la misma prioridad.
- La prioridad se asignara con la sucesion de fibonacci, el nivel mas bajo, va a ser 1, y el mas alto corresponde a 21.
#### Variables Claves
- Puntos Historia asignados a cada tarea
- Prioridad asignada a cada tarea
- Variable de decision si se toma o no la tarea
#### Parametros
- Prioridad asignada a cada tarea
#### Relaciones
Las relaciones entre las Variables y los Parametros es de maximizacion de la suma del producto de las prioridades y la variable de desicion.
#### Tipo de Modelo
Es un modelo de organizacion de naturaleza lineal mixta. (MIP)

### Representacion Matematica
#### Set de Tareas
$$ T = \{T_1,T_2,T_3,T_4,T_5,...,T_N\}$$
#### Indices
$$ \forall i \in T$$
#### Parametros
$$ \forall prioridad_i \in T$$
#### Variable de desicion 
$$ x_i \in \{0,1\}$$
#### Relaciones de las Variables y Parametros
##### Funcion Objetivo
$$ MAX(\sum_{i\in T} x_i \cdot prioridad_i)$$
##### Restricciones
$$ \sum_{i\in T} x_i \cdot puntosH_i \leq 52$$

### Parte B
En este caso, el modelo sera casi igual, lo unico que cambiara sera que ahora se presentara un modelo no para un equipo si para cada desarrollador y esto se logra alterando las restricciones del problema en el apartado de las 'Relaciones'.

### Representacion Matematica
#### Set de Tareas
$$ T = \{T_1,T_2,T_3,T_4,T_5,...,T_N\}$$

$$ D = \{D_1,D_2,D_3,...,D_N\}$$
#### Indices
$$ \forall i \in T$$

$$\forall j \in D$$
#### Parametros
$$ \forall prioridad_i \in T$$
#### Varaible de desicion 
$$ x_{ij}\in \{0,1\}$$
#### Relaciones de las Variables y Parametros
##### Funcion Objetivo
$$ MAX(\sum_{i\in T} \sum_{j\in D} x_{ij} \cdot prioridad_i)$$
##### Restricciones
$$ \sum_{i\in T} x_{ij} \cdot puntosH_i \leq 13 \ \forall j \in D$$

$$ \sum_{j\in D} x_{ij} \leq 1 \ \forall i \in T$$


En este caso, se tendria la consideracion adicional que el modelo se corre por cada desarrollador, es decir, si hay 4 desarrolladores, el modelo se corre 4 veces.

### Parte C (Codigo)

In [ ]:
from pyomo.environ import *
tareas = {
    'T1': {'prioridad': 5, 'puntosH': 8},
    'T2': {'prioridad': 13, 'puntosH': 21},
    'T3': {'prioridad': 3, 'puntosH': 5},
    'T4': {'prioridad': 8, 'puntosH': 13},
    'T5': {'prioridad': 21, 'puntosH': 34},
    'T6': {'prioridad': 2, 'puntosH': 3},
    'T7': {'prioridad': 1, 'puntosH': 2},
    'T8': {'prioridad': 21, 'puntosH': 30}
}

model_A = ConcreteModel()
model_A.T = Set(initialize=tareas.keys())
model_A.x = Var(model_A.T, within=Binary)

model_A.obj = Objective(expr=sum(model_A.x[i] * tareas[i]['prioridad'] for i in model_A.T), sense=maximize)

model_A.restriccion_puntos = Constraint(expr=sum(model_A.x[i] * tareas[i]['puntosH'] for i in model_A.T) <= 52)

solver = SolverFactory('glpk')
solver.solve(model_A)

print("\nResultados - Parte A (Equipo Completo):")
for i in model_A.T:
    if model_A.x[i].value == 1:
        print(f"Tarea {i} seleccionada")

desarrolladores = ['D1', 'D2', 'D3', 'D4']
model_B = ConcreteModel()
model_B.T = Set(initialize=tareas.keys())
model_B.D = Set(initialize=desarrolladores)
model_B.x = Var(model_B.T, model_B.D, within=Binary)

model_B.obj = Objective(expr=sum(model_B.x[i, j] * tareas[i]['prioridad'] for i in model_B.T for j in model_B.D), sense=maximize)

model_B.restriccion_puntos = ConstraintList()
for j in model_B.D:
    model_B.restriccion_puntos.add(sum(model_B.x[i, j] * tareas[i]['puntosH'] for i in model_B.T) <= 13)

model_B.restriccion_unico = ConstraintList()
for i in model_B.T:
    model_B.restriccion_unico.add(sum(model_B.x[i, j] for j in model_B.D) <= 1)

solver.solve(model_B)

print("\nResultados - Parte B (Por Desarrollador):")
for j in model_B.D:
    print(f"\nDesarrollador {j}:")
    for i in model_B.T:
        if model_B.x[i, j].value == 1:
            print(f"  - Tarea {i}")


Resultados - Parte A (Equipo Completo):
Tarea T1 seleccionada
Tarea T4 seleccionada
Tarea T8 seleccionada

Resultados - Parte B (Por Desarrollador):

Desarrollador D1:
  - Tarea T4

Desarrollador D2:
  - Tarea T1
  - Tarea T3

Desarrollador D3:
  - Tarea T6
  - Tarea T7

Desarrollador D4:


## Problema 2

# Problema 2: Asignacion de trabajos a trabajadores

## Parte A

### Problema

Se tiene un conjunto de trabajos que deben ser asignados a un grupo de trabajadores. Cada trabajo requiere un tiempo específico para completarse y genera una ganancia fija al finalizarse. Cada trabajador tiene una disponibilidad limitada de horas. El objetivo es maximizar la ganancia total obtenida por los trabajos asignados, sin que ningún trabajador exceda su disponibilidad horaria

### Suposiciones

- Los trabajos son independientes entre si.
- Cada trabajo puede ser asignado a lo sumo a un trabajador (no se permite que un mismo trabajo sea realizado por varios trabajadores).
- Un trabajo debe ser realizado por completo por el trabajador. No se hacen fracciones de trabajo.
- El tiempo requerido para cada trabajo es fijo y no depende el trabajador.

**Conjuntos:** Trabajo y empleados
$$E = \{E_1, E_2, E_3\}$$
$$T = \{T_1, T_2, \dots, T_5\}$$

### Parametros

- Horas de cada tarea : $$H_t, \quad \forall t \in T$$
- Ganacias de cada tarea : $$G_t, \quad \forall t \in T$$
- Disponibildad de cada empleado : $$D_e, \quad \forall e \in E$$

### Variables clave
Variable de decision binaria: $$X(t,e)$$
Donde indica si ese trabajador hace esa tarea con valores de 1 y 0


### Relaciones

**Función Objetivo:**
$$\text{Maximizar} \sum_{e \in E} \sum_{t \in T} G_t \times X_(t,e)$$
**Restricciones:**
$$\sum_{t \in T} H_t \times X_(t,e) \leq D_e \quad \forall e \in E $$
Restriccion de disponibilidad de empleado

$$\sum_{e \in E} X_(t,e) \leq 1 \quad \forall t \in T $$
Restriccion de asignacion unica de tarea (cada tarea solo se puede asignar una vez)


### Tipo de modelo: MIP

## Parte B
**Conjuntos:** Trabajo y empleados
$$E = \{E_1, E_2, E_3\}$$
$$T = \{T_1, T_2, \dots, T_5\}$$

### Parametros

- Horas de cada tarea : $$H_t, \quad \forall t \in T$$
- Ganacias de cada tarea : $$G_t, \quad \forall t \in T$$
- Disponibildad de cada empleado : $$D_e, \quad \forall e \in E$$

### Variables clave
Variable de decision binaria: $$X(t,e)$$
Donde indica si ese trabajador hace esa tarea con valores de 1 y 0


### Relaciones

**Función Objetivo:**
$$\text{Maximizar} \sum_{e \in E} \sum_{t \in T} G_t \times X_(t,e)$$
**Restricciones:**
$$\sum_{t \in T} H_t \times X_(t,e) \leq D_e \quad \forall e \in E $$
Restriccion de disponibilidad de empleado

$$\sum_{e \in E} X_(t,e) \leq 1 \quad \forall t \in T $$
Restriccion de asignacion unica de tarea (cada tarea solo se puede asignar una vez)

- Solo el trabajador 1 puede realizar el Trabajo 1 
- El Trabajo 3 no puede ser realizado por el trabajador 2
 
Esto hace que al modelo de la parte A se agreguen las siguientes restricciones

$$X_(1,1) \text{ = 1, }$$

$$X_(1,2) \text{ = 0, }$$

$$X_(1,3) \text{ = 0 } $$

$$ X_(3,2) \text{ = 0 } $$

## Problema 3

### Parte A:
#### Indentificacion del problema:
Se quiere 'maximizar' el valor de los recursos enviados en los 3 aviones, teniendo en cuenta que cada avion tiene restricciones de peso, volumen y algunas restricciones especiales entre recursos.
#### Suposiciones:
- No se pueden dividir los recursos entre aviones
- El peso y volumen de los recursos no varia entre aviones 
- Cada avion puede llevar mas de un recurso, excepto la combinacion de equipos medicos y agua
- Los Recursos van a tomar un indice de 1 a 5 en orden
#### Variables Claves
- Peso en toneladas
- Valor del recurso
- Volumen en $m^3$
- Variable desicion
#### Parametros
- Valor del recurso
#### Relaciones
Las relaciones entre las Variables y los Parametros es de maximizacion de la suma del producto de los valores y la variable de desicion.
#### Tipo de Modelo
Es un modelo de organizacion de naturaleza lineal mixta. (MIP)

### Representacion Matematica
#### Set de Tareas
$$ R = \{R_1,R_2,R_3,R_4,R_5\}$$
$$ A = \{A_1,A_2,A_3\}$$
#### Indices
$$ \forall i \in R$$
$$ \forall j \in A$$
#### Parametros
$$ \forall valor_i \in R$$
#### Variable de desicion 
$$ x_{ij} \in \{0,1\}$$
#### Relaciones de las Variables y Parametros
##### Funcion Objetivo
$$ MAX(\sum_{i\in R} \sum_{j\in A} x_{ij} \cdot valor_i)$$
##### Restricciones
$$ \sum_{i\in R} x_{ij} \cdot Peso_i \leq Peso_j \ \forall j \in A$$
$$ \sum_{i\in R} x_{ij} \cdot Volumen_i \leq Volumen_j \ \forall j \in A$$
$$ \sum_{j \in A} x_{ij} \leq 1 \ \forall i \in R$$

- Seguridad Medicamentos

$$ x_{21} = 0$$

- Compatibilidad equipos medicos y agua

$$ j_1 / x_{3j_1} = 1 \wedge j_2 /  x_{4j_2} = 1 \rightarrow j_1 \neq j_2$$


